In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D

In [ ]:
# Load the pre-trained VGG19 model (excluding the top classification layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add BatchNormalization after each convolutional layer
x = base_model.output
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

# Create a custom classification head for your 10 classes
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)

# Create a new model combining the base VGG19 and custom head
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate = 0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define data augmentation for validation/testing
val_datagen = ImageDataGenerator(rescale=1./255)

# Load your dataset (replace 'train_dir' and 'val_dir' with your dataset directories)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8950 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Epoch 1/10
280/280 [==============================] - 5692s 20s/step - loss: 1.6403 - accuracy: 0.4356 - val_loss: 1.0722 - val_accuracy: 0.6492
Epoch 2/10
279/280 [============================>.] - ETA: 15s - loss: 1.1886 - accuracy: 0.5921

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Models/vgg19_transfer_learning_model_1.h5')